## Run Functional Files

In [1]:
%run 1_Function_Master.ipynb 
%run 2_Parameterization.ipynb

## Printing Parameterization

In [2]:
print_parameterization(param)

╒════════════════════════════════════════╤═══════════════╤═════════════════╤═══════════════╤══════════════════════╕
│ V_max = 1.4                            │ λ_P = 0.05    │ μ_V = 0.5       │ μ_P = 0.4     │ K_I = 1.0            │
├────────────────────────────────────────┼───────────────┼─────────────────┼───────────────┼──────────────────────┤
│ γ_Z = 0.4                              │ λ_Z = 0.05    │ μ_V_prime = 1.0 │ μ_delta = 0.4 │ K_h = 1.0            │
├────────────────────────────────────────┼───────────────┼─────────────────┼───────────────┼──────────────────────┤
│ φ = 110236.2205                        │ λ_Z_hat = 0.1 │ μ_u = 0.0       │ μ_g = 0.4     │ K_P = 2.8            │
├────────────────────────────────────────┼───────────────┼─────────────────┼───────────────┼──────────────────────┤
│ g = 1.4                                │ λ_E = 0.5     │ μ_r = 0.5       │ μ_Z = 0.4     │ I_0 = e - 1 = 1.7183 │
├────────────────────────────────────────┼───────────────┼──────────────

## Running Simulations

### Defining Criteria for the Numerical Integrartion Scheme

In [3]:
# Defining length of time span
t_span = 500 # days

# How many simulations desired 
points = 200 # i.e. 200 different total N levels

# In what range of total N
N_t = np.logspace(-4, 1, points) * 2 # 2e-4 - 20

# Numerical integration specifics
max_step   = .1
method     = 'LSODA'
tolerances = [1e-8, 1e-8, 1e-12, 1e-8, 1e-8, 1e-10, 1e-10]

### Running Simulations - Loop Over Total N Values

In [4]:
%%time
# ^ 200 solutions should take about 7 seconds to generate

# Initializing an empty list that holds the generated solutions
sols = []

# Initial conditions of state variables that don't change
Nn_0 = 0.
Pi_0 = 0.
Vi_0 = 0.

# Running loop over desired total N values
for Nt_0 in N_t:
    
    # Define initial phytoplankton and zoolplankton 
    # concentrations as percentages of given total N
    Pu_0 = .1 * Nt_0  
    Z_0  = .2 * Nt_0

    # Initialize free viral abundance based off
    # initial phytoplankton concentration
    r = 1/10 # ratio of hosts:free-virus 
    Ve_0 = est_viral_abund(r, Pu_0, alpha, beta, S_ind, V_ind)

    # Define nutrient pool as remaining concentration
    Nr_0 = Nt_0 - (Pu_0 + Z_0 + Ve_0)

    # Define initial condition array to pass to numerical integrator
    z0 = [Nn_0, Nr_0, Pu_0, Pi_0, Z_0, Vi_0, Ve_0]

    # Run numerical integration scheme
    solution = solve_ivp(model, [0, t_span], z0, args=param,
                         max_step=max_step, method=method, atol=tolerances)     
    
    # Returns list of initial conditions + solution object
    sols.append((z0, solution))

len(sols)

CPU times: user 6.88 s, sys: 147 ms, total: 7.03 s
Wall time: 7.07 s


200

In [5]:
''' Store solutions just incase notebook crashes or to use in other files. '''

%store sols 

Stored 'sols' (list)


## Visualization of Results

In [6]:
''' Read solutions into notebook if kernel restarts. '''

%store -r sols

### Simplistic Animation of Solutions

In [7]:
%matplotlib tk
# ^ Runs code in pop-up window, as sometimes anaimations don't work in jupyter lab setting.

# Defining solution array (List of Lists) for time and each state variable.
# Like a loop of plots, you loop through the list of solutions for each total N level.
time = [(z0, solution.t) for z0, solution in sols]

Nn = [solution.y[0] for z0, solution in sols]
Nr = [solution.y[1] for z0, solution in sols]
PU = [solution.y[2] for z0, solution in sols]
PI = [solution.y[3] for z0, solution in sols]
Z  = [solution.y[4] for z0, solution in sols]        
VI = [solution.y[5] for z0, solution in sols]
VE = [solution.y[6] for z0, solution in sols]

# Initializing figure and subplots
fig = plt.figure(1, figsize=(12, 7))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

# Defining animation function to specifications of matplotlib.animation
def animate(i):
    ax.clear()
    ax2.clear()
    
    for a in [ax, ax2]:
        a.plot(time[i][1], Nn[i], color="darkviolet", label = "N_n")
        a.plot(time[i][1], Nr[i], color="blue"      , label = "N_r")
        a.plot(time[i][1], PU[i], color="aquamarine", label = "P_U")
        a.plot(time[i][1], PI[i], color="limegreen" , label = "P_I")
        a.plot(time[i][1], Z[i] , color="darkgreen" , label = "Z ")
        a.plot(time[i][1], VI[i], color="red"       , label = "V_I")
        a.plot(time[i][1], VE[i], color="darkred"   , label = "V_E")
    
        a.set(xlabel = 'Time (Days)', ylabel = r'Nutrients ($mmol/m^3$)')
        a.grid(which='both', alpha=.3)
        a.minorticks_on()
        a.legend(loc=(.98, 0.05), framealpha=1)
    
    # Linear Plot Settings
    title = (f'Total N: {sum(time[i][0]):.4}, '
             f'for Nn_0 = {time[i][0][0]:.4}, '
             f'Nr_0 = {time[i][0][1]:.4}, '
             f'Pu_0 = {time[i][0][2]:.4}, '
             f'Pi_0 = {time[i][0][3]:.4}, '
             f'Z_0 = {time[i][0][4]:.4}, '
             f'Vi_0 = {time[i][0][5]:.4}, '
             f'Ve_0 = {time[i][0][6]:.4}\n\n'
             'System Dynamics - Linear Scale')

    ax.set_title(title, pad=10)

    # Log Plot Settings
    ax2.set_title('System Dynamics - Log Scale')
    ax2.set_yscale('log')
    ax2.set_ylim(1e-12, 30)

fig.tight_layout(pad=6)
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(sols))
plt.show()

### Complex Animaton of Solutions

The code block below can be used instead of the one above to generate animations plots with key metrics printed beneath the figures. Metrics might include: varible parameterization, final state values at end time, key integration scheme settings, etc. 

In [8]:
# ''' This code block holds useful parameters for printing on plots. '''

# param_labels_fp = ['V_max', 'γ_Z', 'φ', 'g', 'ν_x', 
#                'λ_P', 'λ_Z', 'λ_Z_hat', 'λ_E', 'δ', 'μ_V', 'μ_V_prime', 'μ_u', 'μ_r', 'μ_s', 
#                'μ_P', 'μ_delta', 'μ_g', 'μ_Z', 'K_N', 'K_I', 'K_h', 'K_P', 'I_0 = e - 1', 'ω']

# param_fp = [f'{param_labels_fp[i]} = {param[i]:.4}' for i in range(len(param))]
# param_fp = np.reshape(param_fp, (5,5)).transpose()

In [9]:
# %matplotlib tk

# '''
# Text spacing can be finicky on first run of annimation; if text is not properly aligned,
# close the annimation window and run again. Double check spacing before writing mp4 animation file.
# '''
# time = [(z0, solution.t) for z0, solution in sols]

# Nn = [solution.y[0] for z0, solution in sols]
# Nr = [solution.y[1] for z0, solution in sols]
# PU = [solution.y[2] for z0, solution in sols]
# PI = [solution.y[3] for z0, solution in sols]
# Z  = [solution.y[4] for z0, solution in sols]        
# VI = [solution.y[5] for z0, solution in sols]
# VE = [solution.y[6] for z0, solution in sols]

# # Initializing figure and subplots
# fig = plt.figure(1, figsize=(12, 15))
# ax = fig.add_subplot(311)
# ax2 = fig.add_subplot(312)
# ax3 = fig.add_subplot(313)

# # Defining animation function to specifications of matplotlib.animation
# def animate(i):
#     ax.clear()
#     ax2.clear()
#     ax3.clear()
    
#     for a in [ax, ax2]:
#         a.plot(time[i][1], Nn[i], color="darkviolet", label = "N_n")
#         a.plot(time[i][1], Nr[i], color="blue"      , label = "N_r")
#         a.plot(time[i][1], PU[i], color="aquamarine", label = "P_U")
#         a.plot(time[i][1], PI[i], color="limegreen" , label = "P_I")
#         a.plot(time[i][1], Z[i] , color="darkgreen" , label = "Z ")
#         a.plot(time[i][1], VI[i], color="red"       , label = "V_I")
#         a.plot(time[i][1], VE[i], color="darkred"   , label = "V_E")
    
#         a.set(ylabel = r'Nutrients ($mmol/m^3$)')
#         a.grid(which='both', alpha=.3)
#         a.minorticks_on()
#         a.legend(loc=(.98, 0.05), framealpha=1)
    
#     # Linear Plot Settings
#     title = (f'Total N: {sum(time[i][0]):.4}, '
#              f'for Nn_0 = {time[i][0][0]:.4}, '
#              f'Nr_0 = {time[i][0][1]:.4}, '
#              f'Pu_0 = {time[i][0][2]:.4}, '
#              f'Pi_0 = {time[i][0][3]:.4}, '
#              f'Z_0 = {time[i][0][4]:.4}, '
#              f'Vi_0 = {time[i][0][5]:.4}, '
#              f'Ve_0 = {time[i][0][6]:.4}\n\n'
#              'System Dynamics - Linear Scale')

#     ax.set_title(title, pad=10)

#     # Log Plot Settings
#     ax2.set_title('System Dynamics - Log Scale')
#     ax2.set_xlabel('Time (Days)')
#     ax2.set_yscale('log')
#     ax2.set_ylim(1e-12, 30)
    
#     # Text Settings
#     ax3.text(0, -1.7, f"Number of simulations: {len(sols)}", weight='bold', transform=ax.transAxes)
    
#     ax3.text(0,-1.8, 'Functional Calls:', weight='bold', transform=ax.transAxes)   
#     ax3.text(0,-1.9, f"Integration Method: {method},", transform=ax.transAxes)
#     ax3.text(.22, -1.9, f"Max time step: {max_step},", transform=ax.transAxes)
#     ax3.text(.38, -1.9, f"Absolute state tolerances: {tolerances}", transform=ax.transAxes)

#     ax3.text(0, -2, f"Final state values:", weight='bold', transform=ax.transAxes)
    
#     ax3.text(0, -2.1,   f"N_n:  {Nn[i][-1]},",  transform=ax.transAxes)
#     ax3.text(.25, -2.1, f"N_r:   {Nr[i][-1]},", transform=ax.transAxes)
#     ax3.text(0, -2.2,    f"P_U:  {PU[i][-1]},",  transform=ax.transAxes)
#     ax3.text(.25, -2.2,  f"P_I:   {PI[i][-1]},", transform=ax.transAxes)
#     ax3.text(0, -2.3,   f"Z:     {Z[i][-1]},",  transform=ax.transAxes)
#     ax3.text(0, -2.4,   f"V_I:   {VI[i][-1]},", transform=ax.transAxes)
#     ax3.text(.25, -2.4, f"V_E:  {VE[i][-1]}",   transform=ax.transAxes)
    
#     ax3.text(0, -2.5, "Parameterization: ", weight='bold', transform=ax.transAxes)

#     table = ax3.table(cellText=param_fp, bbox=[0, -.2, 1, .5], cellLoc='left')
#     table.auto_set_font_size(False)
#     table.set_fontsize(9)
#     table.scale(.7, 2)
    
#     # Generate room for text without outlines
#     ax3.get_xaxis().set_visible(False)
#     ax3.get_yaxis().set_visible(False)
#     ax3.spines['top'].set_visible(False)
#     ax3.spines['right'].set_visible(False)
#     ax3.spines['bottom'].set_visible(False)
#     ax3.spines['left'].set_visible(False)

# fig.tight_layout(pad=8)
# ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(sols))
# plt.show()

## File storage

In [13]:
%%time

""" 
Store mp4 file of annimations directly to specified directory; 
Can take a while for large simulation set (takes longer to run simulations
themselves than to save as mp4, 22 seconds for 200 simulations). 

The mp4 format is much more versitile when viewing annimations later on. 
You can easily pause and investigate each solution, unlike with the matplotlib.animation tool.
"""

### SET FILE NAME BEFORE RUNNING 

# Define directory and file name
directory = '/Users/jholmes/Desktop/NCAR/Simulation Annimations/Fixed Initial Condition Ratio/'
file_name = 'Simulations.mp4'

# Save as mp4 to given location
writermp4 = matplotlib.animation.FFMpegWriter(fps=60)
# ani.save(directory + file_name, writer=writermp4) # <- uncomment when ready

CPU times: user 22 s, sys: 409 ms, total: 22.4 s
Wall time: 23.4 s
